#### Read csv

In [2]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv('data/test.csv')

In [4]:
df.columns

Index(['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Author Keywords', 'Index Keywords',
       'Editors', 'Publisher', 'ISSN', 'ISBN', 'CODEN', 'PubMed ID',
       'Language of Original Document', 'Document Type', 'Source', 'EID'],
      dtype='object')

In [7]:
# Function to extract IDs between parentheses
def extract_ids(text):
    return re.findall(r'\((\d+)\)', text)

In [30]:
def duplicate_rows_by_author(df):
    new_df = pd.DataFrame(columns=["author_id", "author_name", "affiliation_full_name", "country"])

    for _, row in df.iterrows():
        # ids = extract_ids(df.loc[i, "Authors with affiliations"])
        # names = df.loc[i, "Authors with affiliations"].split(';')
        authors_with_ids = row["Author full names"].split(';')
        authors_with_affiliations = row["Affiliations"].split(';')
        ids, author_names, affiliations, countries = [], [], [], []
        
        for auth_id, aff in zip(authors_with_ids, authors_with_affiliations):
            # extract author name
            author_name = auth_id.split('.')[0].strip()

            # extract id
            match = re.search(r'\((\d+)\)', auth_id)
            if match:
                extracted_id = match.group(1)
            else:
                continue

            # extract country
            country = aff.rsplit(',', 1)[1].strip().lower()
            if country not in ["morocco", "maroc"]:
                continue

            ids.append(extracted_id)
            author_names.append(author_name)
            affiliations.append(aff)
            countries.append(country)

        data = pd.DataFrame({
            "author_id": ids,
            "author_name": author_names,
            "affiliation_full_name": affiliations,
            "country": countries
        })

        new_df = pd.concat([new_df, data], ignore_index=True)
        
    return new_df

In [32]:
trans_df = duplicate_rows_by_author(df)
trans_df.to_csv("metadata.csv", index=False)
trans_df

,author_id,author_name,affiliation_full_name,country
0,23004567900,"Khachane, M","Laboratoire de Chimie du Solide Minéral, dépar...",morocco
1,6701835701,"Villain, S",Laboratoire de physique du Solide et couches ...,morocco
2,12782205700,"Mjahed, Mostafa (12782205700)","Ecole Royale de l'Air, Maths and Systems Dept....",morocco
3,56698346700,"De Miera Sáenz, L","Departement de Biologie, Université Ibn Tofaï...",morocco
4,56615820600,"El Bahi, S","Mohammed v University, Faculty of Sciences, Ra...",morocco
...,...,...,...,...
150,23978044400,"Elghazi, Haddou (23978044400)","Solid State Physics Laboratory, Faculty of Sci...",morocco
151,7006857659,"Jorio, Anouar (7006857659)","Regional Center of Interface, Sidi Mohammed B...",morocco
152,12239828500,"Ouarsal, Rachid (12239828500)",Laboratoire DIngénierie des Matériaux Organom...,morocco
153,8415913200,"El Bali, Brahim (8415913200)","Department of Chemistry, Faculty of Sciences,...",morocco
